<a href="https://colab.research.google.com/github/jennifer060697/AI15_Project/blob/main/n212_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

마크다운 연습겸 노트 아예 베끼기~

*AIB / SECTION2 / SPRINT1 / NOTE2*
============================

------------------------------
# Generalization
<br/>


# 🏆 학습 목표
- 일반화란 무엇인지 이해하고 다양한 모델 검증 방법으로 일반화 성능..
- HoldOut Balidation, Cross Validation (홀드아웃은 검증 성능이 좀 떨어지긴하지만 일단 빨라서 좋다. cv 는 너무 오래걸리는 문제가 생김. fit 을 여러번 시켜야하니까..)
- 편향/분산 Trade Off

## Warm up

------------------------------------------------------------

## 1. Generalizaion
일반화 (Generalization) 모델을 만드는 데 사용된 분포와 ***동일한 분포***에서 추출한. 이전에는 볼 수 없었던 *새로운 데이터*에 적절하게 적응하는 모델 능력  
머신러닝의 목적은 일반화가 잘된 예측 모델을 만드는것..

## 2. Data Split & Validation (확인 = 검증 세트)
### Hold Out

2way holdout : train, test set  
test set 가 의미있기 위한 2가지 조건  
- 의미있는 결과를 낼 수 있을 정도로 데이터가 충분한가? 이건 어떻게 알아... 1000개로 충분한지, 10000개로 충분한지, 100만개도 부족할수도 있잖아
- 테스트 데이터가 전체 데이터를 대표할 수 있는가? 훈련셋과 특성이 비슷해야한다.

<br/>

보지 않은 데이터! 데이터 리키지! 일반화 성능 측정! 매우 중요하다  

<br/>

3way holdout : train, val, test set

## Case Study : Predicting House Price

In [1]:
# import
import pandas as pd

In [2]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_train.csv')
df_t = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_test.csv')

In [3]:
# train 75%로 분할
len(df)*0.75

1095.0

In [4]:
# pandas sample로 쪼개기
train = df.sample(frac=0.75, random_state=1)
test = df.drop(train.index)

In [5]:
print('train set : ', train.shape)
print('test set : ', test.shape)

train set :  (1095, 81)
test set :  (365, 81)


In [7]:
# validation set 분할

train_2 = train.sample(frac = 0.75, random_state = 1)
val = train.drop(train_2.index)

In [8]:
print('train set : ',train_2.shape)
print('validation set : ', val.shape)
print('test set : ', test.shape)

train set :  (821, 81)
validation set :  (274, 81)
test set :  (365, 81)


In [10]:
from sklearn.model_selection import train_test_split

#train, test, validation
train_sk, test_sk = train_test_split(df, test_size= 0.25, random_state = 1)
train_sk, val_sk = train_test_split(train_sk, test_size= 0.25, random_state = 1)

print('train_sk : ', train_sk.shape)
print('val_sk : ', val_sk.shape)
print('test_sk : ', test_sk.shape)

train_sk :  (821, 81)
val_sk :  (274, 81)
test_sk :  (365, 81)


In [11]:
train_2.index

Int64Index([ 361, 1172,  939,  374, 1382,  453,  114,  584,  529, 1253,
            ...
             493,  168,  104,  255,  304, 1318, 1063, 1316, 1179,  434],
           dtype='int64', length=821)

In [12]:
train_sk.index

Int64Index([ 536,  144, 1123,  395, 1282,  684,  271, 1158,  913,  672,
            ...
            1060,  488,  205,  622,  630, 1234, 1275,  973,  326,  829],
           dtype='int64', length=821)

In [14]:
# 선형적 상관관계가 높은 특성만 사용

num_features = df.dtypes[df.dtypes!='object'].index # 이렇게 뽑을 수 있구만..
df[num_features].corr()['SalePrice'].sort_values(ascending = False).head(10)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64

In [16]:
df[num_features].corr()['SalePrice'].sort_values(ascending = False).tail(10) # 음의 상관관계도 절대값으로 따져볼 수 있으니까 확인해봄.

BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64

In [26]:
features = list(df[num_features].corr()['SalePrice'].sort_values(ascending = False).index[1:8])
print(features)
del features[3]
print(features)

target = 'SalePrice'

# X,y split
def xy_split(df) :
  X = df[features]
  y = df[target]
  return X,y

X_train, y_train = xy_split(train_sk)
X_val, y_val = xy_split(val_sk)
X_test, y_test = xy_split(test_sk)

['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath']
['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', '1stFlrSF', 'FullBath']


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
pd.options.display.float_format = '{:,.2f}'.format

def eval_models(y_true, y_pred) :
  mse = mean_squared_error(y_true, y_pred).round(3)
  rmse = np.sqrt(mse).round(3)
  mae = mean_absolute_error(y_true, y_pred).round(3)
  r2 = r2_score(y_true, y_pred)

  return mse, rmse, mae, r2

baseline = [y_train.mean()] * len(y_train)
eval_models(baseline, y_train)
